In [1]:
import sys
path2cpp_pkg = "/Users/mariusmahiout/Documents/repos/ising_core/build"
sys.path.append(path2cpp_pkg)
import ising

import os
os.chdir("/Users/mariusmahiout/Documents/repos/ising_core/python")
import src.misc_plotting as plotting
import src.utils as utils
import src.model_eval as eval
import src.isingfitter as fitter

import matplotlib.pyplot as plt
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [2]:
def plot_generalized(layout_spec):
    
    num_rows = max([r for (r,_) in layout_spec.keys()])
    num_cols = max([c for (_,c) in layout_spec.keys()])
    
    fig = make_subplots(rows=num_rows, cols=num_cols, horizontal_spacing=0.1, vertical_spacing=0.1)

    curve_color = 'rgba(250, 150, 0, 1)'
    fill_color = 'rgba(250, 150, 0, 0.5)'


    for (r, c), specs in layout_spec.items():

        data = specs['data']
        label = specs['label']

        if isinstance(data[0], float):
            fig.add_trace(go.Scatter(x=steps, y=data, mode='lines', name=f'{label}', line=dict(color=curve_color)), row=r, col=c)

        else:
            data = [a.flatten() for a in data]

            av_data = np.mean(data, axis=1)
            min_data = np.min(data, axis=1)
            max_data = np.max(data, axis=1)
            steps = list(range(len(av_data)))

            fig.add_trace(go.Scatter(x=steps, y=av_data, mode='lines', name=f'Av. {label}', line=dict(color=curve_color)), row=r, col=c)
            fig.add_trace(go.Scatter(x=steps, y=min_data, mode='lines', line=dict(width=0), hoverinfo='skip'), row=r, col=c)
            fig.add_trace(go.Scatter(x=steps, y=max_data, mode='lines', fill='tonexty', fillcolor=fill_color, line=dict(width=0), hoverinfo='skip'), row=r, col=c)

        fig.update_xaxes(title_text="Step", row=r, col=c)
        fig.update_yaxes(title_text=label.capitalize(), row=r, col=c)

    fig.update_layout(height=400 * num_rows, width=400 * num_cols, showlegend=False)

    fig.show()
    #display(fig)

def get_metadata(
    num_units,
    is_empirical_analysis,
    eq_inv_methods=[],
    neq_inv_methods=[],
    **kwargs
):
    metadata = {}
    metadata["num_units"] = num_units

    if is_empirical_analysis:
        bin_width = kwargs['bin_width']
        num_bins = kwargs['num_bins']
        metadata['bin_width'] = bin_width
    else:
        num_sims = kwargs['num_sims']
        num_burn = kwargs.get('num_burn', 1000)
        true_fields = kwargs['true_fields']
        true_couplings = kwargs['true_couplings']
        metadata["true_model"] = {
            'true_fields' : true_fields,
            'true_couplings' : true_couplings,
            'num_sims' : num_sims,
            'num_burn' : num_burn,
        }

    if (eq_inv_methods != []) or (neq_inv_methods != []):
        metadata["inverse_methods"] = {
            'EQ' : eq_inv_methods,
            'NEQ' : neq_inv_methods,
        }
        if ('ML' in eq_inv_methods) or ('ML' in neq_inv_methods):
            # each can be dict if multiple ML models with different hyperparams
            num_steps = kwargs['num_steps']
            learning_rate = kwargs['learning_rate']
            is_converged = kwargs['is_converged']
            metadata['maximum_likelihood'] = {
                'num_steps' : num_steps,
                'learning_rate' : learning_rate,
                'is_converged' : is_converged,
            }
        if ('ML' in eq_inv_methods):
            num_sims_ml = kwargs['num_sims_ml']
            num_burn_ml = kwargs.get('num_burn_ml', 1000)
            metadata['maximum_likelihood']['num_sims'] = num_sims_ml
            metadata['maximum_likelihood']['num_burn'] = num_burn_ml
    return metadata



In [53]:
num_units = 60
num_sims = 30_000
num_burn = 1000

In [54]:
##############
# SIMULATION #
##############

# setting up model
beta = 1.3
h = np.random.uniform(-.3 * beta, .3 * beta, num_units)
J = np.random.normal(0,  beta / np.sqrt(num_units), (num_units, num_units))
for i in range(num_units):
    J[i, i] = 0
    for j in range(i+1, num_units):
        J[j, i] = J[i, j]

true_model = ising.EqModel(J, h)

# simulating
true_sim = true_model.simulate(num_sims, num_burn)

# Gradient ascent

In [55]:
num_sims = 30_000
num_burn = 1000
lr = 0.1
win_size = 10
tol_ml = 1e-3
tol_pl = 1e-5
max_steps = 3000

In [56]:
##############
# LIKELIHOOD #
##############

# setting up model
h_init = np.random.uniform(-1.5, 1.5, num_units)
J_init = np.random.normal(0,  1,  (num_units, num_units))
J_init = (J_init.T + J_init) * np.sqrt(2) / 2
np.fill_diagonal(J_init, 0)

ml_model = ising.EqModel(J_init, h_init)

ml_fitter = fitter.EqFitter(ml_model)
#ml_fitter.TAP(true_sim)

# inference
ml_fitter.maximize_likelihood(
    use_adam=False,

    sample=true_sim, 
    max_steps=max_steps, 
    learning_rate=lr,
    win_size = win_size,
    tolerance= tol_ml, 
    num_sims=num_sims, 
    num_burn=num_burn,
    #calc_llh=True
)


In [57]:
layout_spec = {
    (1, 1): {'data': ml_fitter.fields_grads, 'label': 'Fields gradient'},
    (2, 1): {'data': ml_fitter.couplings_grads, 'label': 'Couplings gradient'},
    (1, 2): {'data': ml_fitter.fields_history, 'label': 'Fields'},
    (2, 2): {'data': ml_fitter.couplings_history, 'label': 'Couplings'},
    (1, 3): {'data': ml_fitter.sd_fields, 'label': 'Fields SD'},   
    (2, 3): {'data': ml_fitter.sd_couplings, 'label': 'Couplings SD'},
    #(3, 2) : {'data': ml_fitter.llhs, 'label': 'LLHs'}
}

plot_generalized(layout_spec)


In [58]:
#####################
# PSEUDO-LIKELIHOOD #
#####################

# setting up model
h_init = np.random.uniform(-1.5, 1.5, num_units)
J_init = np.random.normal(0,  1,  (num_units, num_units))
J_init = (J_init.T + J_init) * np.sqrt(2) / 2
np.fill_diagonal(J_init, 0)

pl_model = ising.EqModel(J_init, h_init)
pl_fitter = fitter.EqFitter(pl_model)
#pl_fitter.TAP(true_sim)

# inference
pl_fitter.maximize_likelihood(
    use_adam=False,

    sample=true_sim, 
    max_steps=max_steps, 
    learning_rate=lr,
    win_size = win_size,
    tolerance= tol_pl, 
    num_sims=0, 
    num_burn=0,
    #calc_llh=True
)


In [59]:
layout_spec = {
    (1, 1): {'data': pl_fitter.fields_grads, 'label': 'Fields gradient'},
    (2, 1): {'data': pl_fitter.couplings_grads, 'label': 'Couplings gradient'},
    (1, 2): {'data': pl_fitter.fields_history, 'label': 'Fields'},
    (2, 2): {'data': pl_fitter.couplings_history, 'label': 'Couplings'},
    (1, 3): {'data': pl_fitter.sd_fields, 'label': 'Fields SD'},
    (2, 3): {'data': pl_fitter.sd_couplings, 'label': 'Couplings SD'},

    #(3, 2) : {'data': pl_fitter.llhs, 'label': 'LLHs'}
}

plot_generalized(layout_spec)


In [60]:
nmf_model = ising.EqModel(J_init, h_init)
nmf_fitter = fitter.EqFitter(nmf_model)
nmf_fitter.naive_mean_field(true_sim)

tap_model = ising.EqModel(J_init, h_init)
tap_fitter = fitter.EqFitter(tap_model)
tap_fitter.TAP(true_sim)


In [61]:
ml_sim = ml_model.simulate(num_sims, num_burn)
pl_sim = pl_model.simulate(num_sims, num_burn)
nmf_sim = nmf_model.simulate(num_sims, num_burn)
tap_sim = tap_model.simulate(num_sims, num_burn)

In [62]:


labels = ["nMF", "TAP", "PLLH", "ML"]
metadata = get_metadata(
    num_units=num_units,
    is_empirical_analysis=False,
    eq_inv_methods=labels,
    num_sims=num_sims,
    true_fields="uniform(-.3 * beta, .3 * beta); beta=1.3",
    true_couplings="normal(0,  beta / sqrt(num_units)); symmetric, beta=1.3",
    num_steps=max_steps,
    learning_rate=lr,
    is_converged=None,
    num_sims_ml=num_sims,
    num_burn_ml=num_burn,
)


def get_analysis_path(analysis_name, num_units, bin_width):
    analysis_path = '../analyses/'
    dir_name = f'n{num_units}b{bin_width}{analysis_name}'
    analysis_path += f'./{dir_name}/'
    return analysis_path


analysis_name = "eq_test"
bin_width = 0
analysis_path = get_analysis_path(analysis_name, num_units, bin_width)

layout_spec = {
    ("fields", "scatter"): (1, 1),
    ("means", "scatter"): (1, 2),
    ("couplings", "scatter"): (2, 1),
    ("pcorrs", "scatter"): (2, 2),
}

ising_eval = eval.IsingEval(
    analysis_path=analysis_path, 
    metadata=metadata,
    true_model=true_model,
    est_models=[nmf_model, tap_model, pl_model, ml_model],
    true_sample=true_sim,
    est_samples=[nmf_sim, tap_sim, pl_sim, ml_sim],
    labels=labels,
    layout_spec=layout_spec
)
ising_eval.generate_plots()

    'data': [{'legendgroup': 'nMF',
              'marker': {'color': 'blue', 's…